In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import image_utils

In [35]:
df=pd.read_csv('94_character_TMNIST.csv')

<b> Data Set : TMNIST Alphabet (94 characters) from  https://www.kaggle.com/datasets/nikbearbrown/tmnist-alphabet-94-characters

In [12]:
y=df['labels']
X=df.drop(['names','labels'],axis=1)
num_classes=df['labels'].nunique()
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.25, random_state=42,stratify=y)
X_train= (X_train.astype('float32'))/255.0
X_test = (X_test.astype('float32'))/255.0
X_train_norm=X_train.values.reshape(-1,28,28,1)
X_test_norm=X_test.values.reshape(-1,28,28,1)

In [51]:
import keras
from keras import layers
import numpy as np

latent_dim = 3
height = 28
width = 28
channels = 1
generator_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(128 * 14 * 14)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((14, 14, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x=layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_12 (Dense)            (None, 25088)             100352    
                                                                 
 leaky_re_lu_54 (LeakyReLU)  (None, 25088)             0         
                                                                 
 reshape_6 (Reshape)         (None, 14, 14, 128)       0         
                                                                 
 conv2d_48 (Conv2D)          (None, 14, 14, 256)       819456    
                                                                 
 leaky_re_lu_55 (LeakyReLU)  (None, 14, 14, 256)       0         
                                                                 
 conv2d_transpose_6 (Conv2DT  (None, 28, 28, 256)      104

In [52]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(learning_rate=0.0008,
    clipvalue=1.0,decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,
    loss='binary_crossentropy')

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_52 (Conv2D)          (None, 26, 26, 128)       1280      
                                                                 
 leaky_re_lu_59 (LeakyReLU)  (None, 26, 26, 128)       0         
                                                                 
 conv2d_53 (Conv2D)          (None, 12, 12, 128)       262272    
                                                                 
 leaky_re_lu_60 (LeakyReLU)  (None, 12, 12, 128)       0         
                                                                 
 conv2d_54 (Conv2D)          (None, 5, 5, 128)         262272    
                                                                 
 leaky_re_lu_61 (LeakyReLU)  (None, 5, 5, 128)         0  

In [53]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input,gan_output)

gan_optimizer = keras.optimizers.RMSprop(learning_rate=0.0004,clipvalue=1.0,
  decay=1e-8)
gan.compile(optimizer=gan_optimizer,loss='binary_crossentropy')

### basic mode

In [49]:
import os
from keras.preprocessing import image

x_train = X_train_norm

iterations = 1000
batch_size = 20
save_dir = 'your_dir'

start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,
        latent_dim))

    generated_images = generator.predict(random_latent_vectors)
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    labels = np.concatenate([np.ones((batch_size, 1)),
        np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)

    d_loss = discriminator.train_on_batch(combined_images, labels)

    random_latent_vectors = np.random.normal(size=(batch_size,
        latent_dim))
    misleading_targets = np.zeros((batch_size, 1))
    a_loss = gan.train_on_batch(random_latent_vectors,
        misleading_targets)

    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0

    if step % 100 == 0:

        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)

        img = image_utils.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir,'generated'\
        +str(step)+'.png'))

        img = image_utils.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir,'real' + str(step) + '.png'))

1/1 [==============================] - 0s 226ms/step
discriminator loss: 0.5819021463394165
adversarial loss: 1.313842535018921
1/1 [==============================] - 0s 287ms/step
discriminator loss: 0.42166367173194885
adversarial loss: 1.2101536989212036
1/1 [==============================] - 0s 275ms/step


1/1 [==============================] - 0s 294ms/step
discriminator loss: 0.31181642413139343
adversarial loss: 2.0841286182403564
1/1 [==============================] - 0s 290ms/step
discriminator loss: 0.4095856249332428
adversarial loss: 1.2451709508895874
1/1 [==============================] - 0s 266ms/step


1/1 [==============================] - 0s 276ms/step
discriminator loss: 0.34231629967689514
adversarial loss: 1.4694606065750122
1/1 [==============================] - 0s 306ms/step


1/1 [==============================] - 0s 268ms/step
discriminator loss: 0.19639469683170319
adversarial loss: 2.4564671516418457
1/1 [==============================] - 0s 282ms/step
discriminator loss: 0.2028581202030182
adversarial loss: 5.436891078948975
1/1 [==============================] - 0s 281ms/step


1/1 [==============================] - 0s 332ms/step
discriminator loss: 0.6436506509780884
adversarial loss: 1.341591715812683
1/1 [==============================] - 0s 306ms/step


1/1 [==============================] - 0s 303ms/step
discriminator loss: 0.08362268656492233
adversarial loss: 14.283622741699219
1/1 [==============================] - 0s 303ms/step
discriminator loss: 0.038433827459812164
adversarial loss: 9.353408813476562
1/1 [==============================] - 0s 284ms/step


1/1 [==============================] - 0s 285ms/step


Because there are fewer steps, the result is not very well. However, it can be seen that the model has learned something and is in the process of progress.

### Change latent tim

In [54]:
import os
from keras.preprocessing import image

x_train = X_train_norm

iterations = 1000
batch_size = 20
save_dir = 'your_dir'


start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,
        latent_dim))

    generated_images = generator.predict(random_latent_vectors)
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    labels = np.concatenate([np.ones((batch_size, 1)),
        np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)

    d_loss = discriminator.train_on_batch(combined_images, labels)

    random_latent_vectors = np.random.normal(size=(batch_size,
        latent_dim))
    misleading_targets = np.zeros((batch_size, 1))
    a_loss = gan.train_on_batch(random_latent_vectors,
        misleading_targets)

    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0

    if step % 100 == 0:

        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)

        img = image_utils.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir,'generated'\
        +str(step)+'.png'))

        img = image_utils.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir,'real' + str(step) + '.png'))

1/1 [==============================] - 0s 317ms/step
discriminator loss: 0.6975625157356262
adversarial loss: 0.6746835112571716
1/1 [==============================] - 0s 281ms/step
discriminator loss: 0.9248441457748413
adversarial loss: 1.4338958263397217
1/1 [==============================] - 0s 304ms/step


1/1 [==============================] - 0s 284ms/step
discriminator loss: 0.43798741698265076
adversarial loss: 1.230730652809143
1/1 [==============================] - 0s 290ms/step
discriminator loss: 0.9489839673042297
adversarial loss: 0.39185652136802673
1/1 [==============================] - 0s 303ms/step


1/1 [==============================] - 0s 288ms/step
discriminator loss: 0.24281270802021027
adversarial loss: 2.0458850860595703
1/1 [==============================] - 0s 304ms/step


1/1 [==============================] - 0s 301ms/step
discriminator loss: 0.2591329514980316
adversarial loss: 2.964339017868042
1/1 [==============================] - 0s 326ms/step
discriminator loss: 0.19052654504776
adversarial loss: 7.379116058349609
1/1 [==============================] - 0s 321ms/step


1/1 [==============================] - 0s 420ms/step
discriminator loss: 0.07081776857376099
adversarial loss: 7.008540153503418
1/1 [==============================] - 0s 440ms/step


1/1 [==============================] - 0s 450ms/step
discriminator loss: -0.020160984247922897
adversarial loss: 12.460616111755371
1/1 [==============================] - 0s 487ms/step
discriminator loss: -0.02794456109404564
adversarial loss: 11.535675048828125
1/1 [==============================] - 0s 486ms/step


1/1 [==============================] - 1s 522ms/step


As a result, changing latent tim to 3 is not a very good choice. However, as mentioned above, because the number of steps is very small, the results are all not particularly ideal.

### Changing the network architecture

In [55]:
latent_dim = 2
height = 28
width = 28
channels = 1
generator_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(128 * 14 * 14)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((14, 14, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x=layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 2)]               0         
                                                                 
 dense_14 (Dense)            (None, 25088)             75264     
                                                                 
 leaky_re_lu_63 (LeakyReLU)  (None, 25088)             0         
                                                                 
 reshape_7 (Reshape)         (None, 14, 14, 128)       0         
                                                                 
 conv2d_56 (Conv2D)          (None, 14, 14, 256)       819456    
                                                                 
 leaky_re_lu_64 (LeakyReLU)  (None, 14, 14, 256)       0         
                                                                 
 conv2d_transpose_7 (Conv2DT  (None, 28, 28, 256)      104

In [56]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(learning_rate=0.0008,
    clipvalue=1.0,decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,
    loss='binary_crossentropy')

Model: "model_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_59 (Conv2D)          (None, 26, 26, 128)       1280      
                                                                 
 leaky_re_lu_67 (LeakyReLU)  (None, 26, 26, 128)       0         
                                                                 
 conv2d_60 (Conv2D)          (None, 12, 12, 128)       262272    
                                                                 
 leaky_re_lu_68 (LeakyReLU)  (None, 12, 12, 128)       0         
                                                                 
 conv2d_61 (Conv2D)          (None, 5, 5, 128)         262272    
                                                                 
 leaky_re_lu_69 (LeakyReLU)  (None, 5, 5, 128)         0  

In [57]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input,gan_output)

gan_optimizer = keras.optimizers.RMSprop(learning_rate=0.0004,clipvalue=1.0,
  decay=1e-8)
gan.compile(optimizer=gan_optimizer,loss='binary_crossentropy')

In [58]:
import os
from keras.preprocessing import image

x_train = X_train_norm

iterations = 1000
batch_size = 20
save_dir = 'your_dir'

start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,
        latent_dim))

    generated_images = generator.predict(random_latent_vectors)
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    labels = np.concatenate([np.ones((batch_size, 1)),
        np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)

    d_loss = discriminator.train_on_batch(combined_images, labels)

    random_latent_vectors = np.random.normal(size=(batch_size,
        latent_dim))
    misleading_targets = np.zeros((batch_size, 1))
    a_loss = gan.train_on_batch(random_latent_vectors,
        misleading_targets)

    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0

    if step % 100 == 0:

        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)

        img = image_utils.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir,'generated'\
        +str(step)+'.png'))

        img = image_utils.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir,'real' + str(step) + '.png'))

1/1 [==============================] - 0s 351ms/step
discriminator loss: 0.6911717057228088
adversarial loss: 0.6796035766601562
1/1 [==============================] - 0s 324ms/step
discriminator loss: 0.5380058884620667
adversarial loss: 0.7852485775947571
1/1 [==============================] - 0s 365ms/step


1/1 [==============================] - 0s 366ms/step
discriminator loss: 0.5065340995788574
adversarial loss: 1.4505994319915771
1/1 [==============================] - 0s 375ms/step
discriminator loss: 0.4674326479434967
adversarial loss: 3.518362522125244
1/1 [==============================] - 0s 367ms/step


1/1 [==============================] - 0s 372ms/step
discriminator loss: 0.6746774911880493
adversarial loss: 2.4719533920288086
1/1 [==============================] - 0s 378ms/step


1/1 [==============================] - 0s 397ms/step
discriminator loss: 0.45327743887901306
adversarial loss: 0.883374035358429
1/1 [==============================] - 0s 408ms/step
discriminator loss: 0.5023770332336426
adversarial loss: 1.5029150247573853
1/1 [==============================] - 0s 407ms/step


1/1 [==============================] - 0s 416ms/step
discriminator loss: 0.45579075813293457
adversarial loss: 0.7993400692939758
1/1 [==============================] - 0s 419ms/step


1/1 [==============================] - 0s 422ms/step
discriminator loss: 0.5309454202651978
adversarial loss: 1.0973360538482666
1/1 [==============================] - 0s 420ms/step
discriminator loss: 0.15114820003509521
adversarial loss: 3.674196720123291
1/1 [==============================] - 0s 375ms/step


1/1 [==============================] - 0s 412ms/step


After reducing two Conv2D layers, we obtained the best result in this experiment.

<b>References:<b>  
[1] : GAN sample : https://blog.csdn.net/qq_42740834/article/details/123692837  
[2] : 3D - Variational Autoencoder implementation Error3D : https://www.likecs.com/ask-8448405.html   
[3] : https://blog.csdn.net/baoxin1100/article/details/107917633

## MIT license

Copyright <2022> Peichen Han

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.